In [ ]:
# 1. Kill any existing Xvfb processes to avoid port conflicts
!fuser -k /tmp/.X11-unix/X1 2>/dev/null || true

# 2. Install/Update dependencies
!apt-get update
!apt-get install -y xvfb x11-utils google-chrome-stable
!pip install pyvirtualdisplay pyautogui python-xlib

from pyvirtualdisplay import Display
import os
import time
import subprocess

# 3. Start the display with a high display number to avoid system conflicts
# We use 'backend="xvfb"' to be explicit
disp = Display(visible=False, size=(1080, 2500), backend="xvfb")
disp.start()

# 4. Sync the environment variable to whatever the display actually picked
os.environ["DISPLAY"] = f":{disp.display}"
print(f"Virtual display started on {os.environ['DISPLAY']}")

# 5. NOW import pyautogui (it must happen AFTER the env var is set)
import pyautogui

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,966 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [69.3 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,411 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,598 kB

In [ ]:
try:
    print(f"Screen size detected: {pyautogui.size()}")
except Exception as e:
    print(f"Still can't connect: {e}")

Screen size detected: Size(width=1080, height=2500)


In [ ]:
# Install the screenshot utility
!apt-get update
!apt-get install -y scrot gnome-screenshot

# Ensure Pillow is the right version
!pip install --upgrade Pillow

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 2s (2,105 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installe

In [ ]:
# 1. Install Google Chrome Stable
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list
!apt-get update
!apt-get install -y google-chrome-stable

# 2. Find the exact path
import subprocess
chrome_path = subprocess.getoutput("which google-chrome")
print(f"Chrome is located at: {chrome_path}")

OK
Get:1 http://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 http://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,212 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 6,954 B in 2s (3,060 B/s)
Reading package lists... Done
W: http://dl.google.com/linux/chrome/deb/dists/stable/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for 

# # Selenium

In [ ]:
# -----------------------------
# 0) Clean up any old browser/driver processes (IMPORTANT)
# -----------------------------
# !pkill -f chrome || true
# !pkill -f chromium || true
# !pkill -f chromedriver || true

# -----------------------------
# 1) Install matched browser + driver
# -----------------------------
!apt-get update -y
!apt-get install -y chromium-browser chromium-chromedriver xvfb
!pip -q install selenium

# -----------------------------
# 2) Start/ensure Xvfb display (use your existing disp if already running)
# -----------------------------
from pyvirtualdisplay import Display
import os, time

# If you already created "disp" earlier and it is running, you can skip these 3 lines.
disp = Display(visible=False, size=(1080, 2500), backend="xvfb")
disp.start()

os.environ["DISPLAY"] = f":{disp.display}"
print("DISPLAY =", os.environ["DISPLAY"])

# -----------------------------
# 3) Selenium launch (NOT headless; uses Xvfb)
# -----------------------------
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

opts = webdriver.ChromeOptions()
opts.binary_location = "/usr/bin/chromium-browser"  # matched to chromium-chromedriver
opts.add_argument("--no-sandbox")
opts.add_argument("--disable-dev-shm-usage")
opts.add_argument("--disable-gpu")
opts.add_argument("--window-size=1080,2500")
opts.add_argument("--disable-features=TranslateUI")
opts.add_argument("--no-first-run")
opts.add_argument("--no-default-browser-check")

driver = webdriver.Chrome(options=opts)
driver.get("https://ronchigram.com/")
WebDriverWait(driver, 30).until(lambda d: d.execute_script("return document.readyState") == "complete")
time.sleep(2)

driver.save_screenshot("debug_loaded.png")
print("Saved debug_loaded.png")

# -----------------------------
# 4) Helpers: set values + click calculate + save only ronchigram
# -----------------------------
import re, base64

def set_value_js(el, value):
    driver.execute_script("""
        const el = arguments[0];
        const value = String(arguments[1]);
        el.focus();

        const proto = Object.getPrototypeOf(el);
        const desc = Object.getOwnPropertyDescriptor(proto, 'value');
        if (desc && desc.set) desc.set.call(el, value);
        else el.value = value;

        el.dispatchEvent(new Event('input', {bubbles:true}));
        el.dispatchEvent(new Event('change', {bubbles:true}));
        el.dispatchEvent(new Event('blur', {bubbles:true}));
    """, el, value)

def text_norm(s): return re.sub(r"\s+", " ", (s or "").strip().lower())

def click_calculate():
    for b in driver.find_elements(By.XPATH, "//button"):
        if "calculate" in text_norm(b.text):
            b.click()
            return True
    for b in driver.find_elements(By.XPATH, "//input[@type='button' or @type='submit']"):
        if "calculate" in text_norm(b.get_attribute("value")):
            b.click()
            return True
    raise RuntimeError("CALCULATE button not found")

def save_ronchigram_only(out_png):
    # Prefer the largest canvas (main ronchigram)
    canvases = driver.find_elements(By.CSS_SELECTOR, "canvas")
    if canvases:
        c = max(canvases, key=lambda e: e.size.get("width",0) * e.size.get("height",0))
        data_url = driver.execute_script("return arguments[0].toDataURL('image/png');", c)
        if isinstance(data_url, str) and data_url.startswith("data:image/png;base64,"):
            b64 = data_url.split(",", 1)[1]
            with open(out_png, "wb") as f:
                f.write(base64.b64decode(b64))
            print("Saved (canvas) ->", out_png)
            return

    # Fallback: screenshot the largest <img>
    imgs = driver.find_elements(By.CSS_SELECTOR, "img")
    if imgs:
        im = max(imgs, key=lambda e: e.size.get("width",0) * e.size.get("height",0))
        im.screenshot(out_png)
        print("Saved (img element) ->", out_png)
        return

    raise RuntimeError("No canvas or img found to export ronchigram")

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

def find_value_widget_after_label(label_text, timeout=20):
    """
    Returns the value-entry widget associated with a label like 'C10:', 'C12:' etc.
    Works even if the widget is not a plain <input> (e.g., contenteditable div / spinbutton).
    """

    # Find an element that contains the label text
    lab = WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located(
            (By.XPATH, f"//*[contains(normalize-space(.), '{label_text}')]")
        )
    )

    # Prefer to search within a reasonable "row" container around that label
    # (Closest ancestor div is usually the row)
    row = lab.find_element(By.XPATH, "ancestor::div[1]")

    # 1) First try normal inputs within the same row
    els = row.find_elements(By.XPATH, ".//input[not(@type='hidden')]")
    if els:
        return els[0]

    # 2) Try contenteditable widgets
    els = row.find_elements(By.XPATH, ".//*[@contenteditable='true']")
    if els:
        return els[0]

    # 3) Try aria spinbutton/textbox-like widgets
    els = row.find_elements(By.XPATH, ".//*[@role='spinbutton' or @role='textbox']")
    if els:
        return els[0]

    # 4) Last resort: search forward in the document (not limited to row)
    els = driver.find_elements(By.XPATH, f"//*[contains(normalize-space(.), '{label_text}')]/following::input[1]")
    if els:
        return els[0]

    els = driver.find_elements(By.XPATH, f"//*[contains(normalize-space(.), '{label_text}')]/following::*[@contenteditable='true'][1]")
    if els:
        return els[0]

    raise RuntimeError(f"Could not find a value widget for label: {label_text}")

# -----------------------------
# 5) Auto-find keV + df fields (same logic you used before)
# -----------------------------
def find_candidate_inputs():
    return driver.find_elements(By.CSS_SELECTOR, "input, textarea, select, [contenteditable='true']")

def describe_element(el):
    return {
        "tag": el.tag_name,
        "type": el.get_attribute("type"),
        "id": el.get_attribute("id"),
        "name": el.get_attribute("name"),
        "placeholder": el.get_attribute("placeholder"),
        "aria": el.get_attribute("aria-label"),
        "class": el.get_attribute("class"),
        "value": el.get_attribute("value"),
    }

def score_for_key(desc, key):
    hay = " ".join([desc.get("id") or "", desc.get("name") or "", desc.get("placeholder") or "",
                    desc.get("aria") or "", desc.get("class") or ""]).lower()
    syn = {
        "keV": ["kev", "beam", "energy", "accelerating"],
        "df_nm": ["df", "defocus", "focus", "nm"],
    }.get(key, [key.lower()])
    score = 0
    for w in syn:
        if w in hay:
            score += 10
    if (desc.get("type") or "").lower() in ["number", "text", ""]:
        score += 2
    return score

def pick_best_field(key):
    scored = []
    for el in find_candidate_inputs():
        d = describe_element(el)
        s = score_for_key(d, key)
        if s > 0:
            scored.append((s, el, d))
    scored.sort(key=lambda x: x[0], reverse=True)
    if scored:
        s, el, d = scored[0]
        print(f"[AutoMatch] {key} score={s} id={d.get('id')} name={d.get('name')} placeholder={d.get('placeholder')} aria={d.get('aria')}")
        return el
    return None

field_keV = pick_best_field("keV")
field_df  = pick_best_field("df_nm")

if field_keV is None or field_df is None:
    driver.save_screenshot("debug_inventory.png")
    print("Could not auto-locate fields. Saved debug_inventory.png")
    # Print inventory to manually hard-code selectors if needed
    cands = find_candidate_inputs()
    for i, el in enumerate(cands[:80]):
        print(i, describe_element(el))
    raise RuntimeError("Fields not found. Use inventory above to hard-code selectors.")

# -----------------------------
# 6) CHANGE VALUES HERE (keV / C10 / C12 / C21 / C23 / C30)
# -----------------------------
keV = 200

# Aberration coefficients (units exactly as shown in the UI)
ab = {
    "C10": 300.0,    # Defocus (Å)
    "C12": -0.849,   # A1  (2-fold astigmatism), nm
    "C21":  1.38,    # B2  (axial coma), nm
    "C23":  1.59,    # A2  (3-fold astigmatism), nm
    "C30": 235.0,    # C3  (3rd order spherical), nm
}

# Existing beam energy
set_value_js(field_keV, keV)

# NEW: bind coefficient fields by visible labels
field_C10 = find_input_after_text("C10:")
field_C12 = find_input_after_text("C12:")
field_C21 = find_input_after_text("C21:")
field_C23 = find_input_after_text("C23:")
field_C30 = find_input_after_text("C30:")

# Set values
set_value_js(field_C10, ab["C10"])
set_value_js(field_C12, ab["C12"])
set_value_js(field_C21, ab["C21"])
set_value_js(field_C23, ab["C23"])
set_value_js(field_C30, ab["C30"])

# Calculate + save ronchigram
click_calculate()
time.sleep(1.5)

save_ronchigram_only(
    f"ronchi_keV{keV}_"
    f"C10{ab['C10']}_C12{ab['C12']}_C21{ab['C21']}_"
    f"C23{ab['C23']}_C30{ab['C30']}.png"
)

print("Done.")

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://dl.google.com/linux/chrome/deb stable InRelease
Get:6 https://cli.github.com/packages stable InRelease [3,917 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,556 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,855 kB]
Fetched 12.6 MB in 3s (4,212 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"following::input[1]"}
  (Session info: chrome=143.0.7499.146); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
#0 0x5d0093c4d9aa <unknown>
#1 0x5d00936822db <unknown>
#2 0x5d00936d382f <unknown>
#3 0x5d00936d3a71 <unknown>
#4 0x5d00936c7a56 <unknown>
#5 0x5d00936c7947 <unknown>
#6 0x5d009371f139 <unknown>
#7 0x5d00936c6112 <unknown>
#8 0x5d00936c6dc1 <unknown>
#9 0x5d0093c16609 <unknown>
#10 0x5d0093c1954d <unknown>
#11 0x5d0093bff331 <unknown>
#12 0x5d0093c1a11b <unknown>
#13 0x5d0093be5ff0 <unknown>
#14 0x5d0093c3ad98 <unknown>
#15 0x5d0093c3af69 <unknown>
#16 0x5d0093c4cd03 <unknown>
#17 0x793d27036ac3 <unknown>


In [ ]:
# -----------------------------
# 0) Clean up any old browser/driver processes (IMPORTANT)
# -----------------------------
# !pkill -f chrome || true
# !pkill -f chromium || true
# !pkill -f chromedriver || true

# -----------------------------
# 1) Install matched browser + driver
# -----------------------------
!apt-get update -y
!apt-get install -y chromium-browser chromium-chromedriver xvfb
!pip -q install selenium pyvirtualdisplay

# -----------------------------
# 2) Start/ensure Xvfb display (use your existing disp if already running)
# -----------------------------
from pyvirtualdisplay import Display
import os, time

disp = Display(visible=False, size=(1080, 2500), backend="xvfb")
disp.start()

os.environ["DISPLAY"] = f":{disp.display}"
print("DISPLAY =", os.environ["DISPLAY"])

# -----------------------------
# 3) Selenium launch (NOT headless; uses Xvfb)
# -----------------------------
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

opts = webdriver.ChromeOptions()
opts.binary_location = "/usr/bin/chromium-browser"  # matched to chromium-chromedriver
opts.add_argument("--no-sandbox")
opts.add_argument("--disable-dev-shm-usage")
opts.add_argument("--disable-gpu")
opts.add_argument("--window-size=1080,2500")
opts.add_argument("--disable-features=TranslateUI")
opts.add_argument("--no-first-run")
opts.add_argument("--no-default-browser-check")

driver = webdriver.Chrome(options=opts)
driver.get("https://ronchigram.com/")
WebDriverWait(driver, 30).until(lambda d: d.execute_script("return document.readyState") == "complete")
time.sleep(2)

driver.save_screenshot("debug_loaded.png")
print("Saved debug_loaded.png")

# -----------------------------
# 4) Helpers: set values + click calculate + save only ronchigram
# -----------------------------
import re, base64

def set_value_js(el, value):
    driver.execute_script("""
        const el = arguments[0];
        const value = String(arguments[1]);
        el.focus();

        const proto = Object.getPrototypeOf(el);
        const desc = Object.getOwnPropertyDescriptor(proto, 'value');
        if (desc && desc.set) desc.set.call(el, value);
        else el.value = value;

        el.dispatchEvent(new Event('input', {bubbles:true}));
        el.dispatchEvent(new Event('change', {bubbles:true}));
        el.dispatchEvent(new Event('blur', {bubbles:true}));
    """, el, value)

def set_value_any(el, value):
    """
    Sets value into either:
      - <input>/<textarea> using value property + input/change events
      - contenteditable/custom widgets using selectAll/insertText or textContent
    """
    tag = (el.tag_name or "").lower()
    if tag in ("input", "textarea", "select"):
        set_value_js(el, value)
        return

    driver.execute_script("""
        const el = arguments[0];
        const value = String(arguments[1]);
        el.focus();

        if (el.isContentEditable) {
            document.execCommand('selectAll', false, null);
            document.execCommand('insertText', false, value);
        } else {
            el.textContent = value;
        }

        el.dispatchEvent(new Event('input', {bubbles:true}));
        el.dispatchEvent(new Event('change', {bubbles:true}));
        el.dispatchEvent(new Event('blur', {bubbles:true}));
    """, el, str(value))

def text_norm(s):
    return re.sub(r"\s+", " ", (s or "").strip().lower())

def click_calculate():
    for b in driver.find_elements(By.XPATH, "//button"):
        if "calculate" in text_norm(b.text):
            b.click()
            return True
    for b in driver.find_elements(By.XPATH, "//input[@type='button' or @type='submit']"):
        if "calculate" in text_norm(b.get_attribute("value")):
            b.click()
            return True
    raise RuntimeError("CALCULATE button not found")

def save_ronchigram_only(out_png):
    canvases = driver.find_elements(By.CSS_SELECTOR, "canvas")
    if canvases:
        c = max(canvases, key=lambda e: e.size.get("width",0) * e.size.get("height",0))
        data_url = driver.execute_script("return arguments[0].toDataURL('image/png');", c)
        if isinstance(data_url, str) and data_url.startswith("data:image/png;base64,"):
            b64 = data_url.split(",", 1)[1]
            with open(out_png, "wb") as f:
                f.write(base64.b64decode(b64))
            print("Saved (canvas) ->", out_png)
            return

    imgs = driver.find_elements(By.CSS_SELECTOR, "img")
    if imgs:
        im = max(imgs, key=lambda e: e.size.get("width",0) * e.size.get("height",0))
        im.screenshot(out_png)
        print("Saved (img element) ->", out_png)
        return

    raise RuntimeError("No canvas or img found to export ronchigram")

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

def find_value_widget_for_Cnm(n, m, timeout=20):
    """
    Finds the value-entry widget for a coefficient Cnm (e.g., C10, C12, C21, C23, C30)
    even when n,m are rendered as subscripts (separate DOM nodes).
    Strategy:
      1) Find an element that contains 'C' and also contains the digits for n and m nearby
      2) From that element, grab the first editable widget after it (input/contenteditable/role)
    """
    # Look for a label-ish node that contains 'C' and the digits somewhere in its textContent.
    # This works for cases like: "C" + "<sub>10</sub>" because textContent becomes "C10".
    key = f"C{n}{m}"

    # candidate nodes: span/label/div/td/th (covers most UI patterns)
    xpath = (
        "//*[self::label or self::span or self::div or self::td or self::th]"
        f"[contains(normalize-space(.), 'C') and contains(normalize-space(.), '{n}{m}')]"
    )

    lab = WebDriverWait(driver, timeout).until(lambda d: d.find_element(By.XPATH, xpath))

    # Now pick the first editable control after this label node
    for xp in [
        "following::input[not(@type='hidden')][1]",
        "following::*[@contenteditable='true'][1]",
        "following::*[@role='spinbutton' or @role='textbox'][1]",
    ]:
        els = lab.find_elements(By.XPATH, xp)
        if els:
            return els[0]

    raise RuntimeError(f"Found label for {key} but could not find its input widget.")

from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.common.alert import Alert

def dismiss_alert_if_present():
    try:
        a = Alert(driver)
        txt = a.text
        a.accept()
        print("Dismissed alert:", txt)
        return True
    except Exception:
        return False
def find_display_px_field():
    # Try common ids/names first (fast)
    for css in ["#dispsize", "#disppx", "#disp_px", "#displaypx", "input[name='dispsize']", "input[name='disppx']"]:
        els = driver.find_elements(By.CSS_SELECTOR, css)
        if els:
            return els[0]

    # Fallback: search all inputs for keywords
    for el in driver.find_elements(By.CSS_SELECTOR, "input"):
        hay = " ".join([
            el.get_attribute("id") or "",
            el.get_attribute("name") or "",
            el.get_attribute("placeholder") or "",
            el.get_attribute("aria-label") or "",
            el.get_attribute("class") or "",
        ]).lower()
        if ("disp" in hay or "display" in hay) and ("px" in hay or "pixel" in hay or "size" in hay):
            return el

    raise RuntimeError("Could not find Display size (pixels) field.")


# -----------------------------
# 5) Auto-find keV field (df field not needed if using C10)
# -----------------------------
def find_candidate_inputs():
    return driver.find_elements(By.CSS_SELECTOR, "input, textarea, select, [contenteditable='true']")

def describe_element(el):
    return {
        "tag": el.tag_name,
        "type": el.get_attribute("type"),
        "id": el.get_attribute("id"),
        "name": el.get_attribute("name"),
        "placeholder": el.get_attribute("placeholder"),
        "aria": el.get_attribute("aria-label"),
        "class": el.get_attribute("class"),
        "value": el.get_attribute("value"),
    }

def score_for_key(desc, key):
    hay = " ".join([
        desc.get("id") or "",
        desc.get("name") or "",
        desc.get("placeholder") or "",
        desc.get("aria") or "",
        desc.get("class") or ""
    ]).lower()
    syn = {
        "keV": ["kev", "beam", "energy", "accelerating"],
    }.get(key, [key.lower()])
    score = 0
    for w in syn:
        if w in hay:
            score += 10
    if (desc.get("type") or "").lower() in ["number", "text", ""]:
        score += 2
    return score

def pick_best_field(key):
    scored = []
    for el in find_candidate_inputs():
        d = describe_element(el)
        s = score_for_key(d, key)
        if s > 0:
            scored.append((s, el, d))
    scored.sort(key=lambda x: x[0], reverse=True)
    if scored:
        s, el, d = scored[0]
        print(f"[AutoMatch] {key} score={s} id={d.get('id')} name={d.get('name')} placeholder={d.get('placeholder')} aria={d.get('aria')}")
        return el
    return None

field_keV = pick_best_field("keV")
if field_keV is None:
    driver.save_screenshot("debug_inventory.png")
    raise RuntimeError("Could not auto-locate keV field. See debug_inventory.png")

# -----------------------------
# 6) CHANGE VALUES HERE (keV / C10 / C12 / C21 / C23 / C30)
# -----------------------------
keV = 200

ab = {
    "C10": 300.0,    # Defocus (Å)
    "C12": -0.849,   # A1  (2-fold astigmatism), nm
    "C21":  1.38,    # B2  (axial coma), nm
    "C23":  1.59,    # A2  (3-fold astigmatism), nm
    "C30": 235.0,    # C3  (3rd order spherical), nm
}

set_value_js(field_keV, keV)

# Bind coefficient widgets by visible labels (FIXED)
field_C10 = find_value_widget_for_Cnm(1, 0)
field_C12 = find_value_widget_for_Cnm(1, 2)
field_C21 = find_value_widget_for_Cnm(2, 1)
field_C23 = find_value_widget_for_Cnm(2, 3)
field_C30 = find_value_widget_for_Cnm(3, 0)

# Set values (use set_value_any so C10 works even if it's not a plain <input>)
set_value_any(field_C10, ab["C10"])
set_value_any(field_C12, ab["C12"])
set_value_any(field_C21, ab["C21"])
set_value_any(field_C23, ab["C23"])
set_value_any(field_C30, ab["C30"])

field_disp_px = find_display_px_field()
set_value_any(field_disp_px, 256)   # must be power of 2
dismiss_alert_if_present()

click_calculate()
time.sleep(1.5)

save_ronchigram_only(
    f"ronchi_keV{keV}_"
    f"C10{ab['C10']}_C12{ab['C12']}_C21{ab['C21']}_"
    f"C23{ab['C23']}_C30{ab['C30']}.png"
)

print("Done.")


Hit:1 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 2s (2,191 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state informa